In [47]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/Tritangent_plane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [48]:
xx = QQ['xx'].0

In [49]:
Q.<alpha> = NumberField(xx^2+3)

In [50]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)

In [51]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [52]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [53]:
line = Line([y, z])

In [54]:
eqn_SE1 = P(cubic_new.subs({b:-(c*c+e*f)/c }) )
sing_cubics_SE1 = P(sing_cubics.subs({b:-(c*c+e*f)/c }))

In [55]:
eqn_SE3 = P(2*eqn_SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)}) )
sing_cubics_SE3 = P(sing_cubics_SE1.subs({d:(3*e*f - c^2 + c*f + c*e)/(2*c)}))

In [56]:
eqn_SE9 = P(eqn_SE3.subs({e : alpha*c}) /c)
sing_cubics_SE9 = P(sing_cubics_SE3.subs({e : alpha*c}))

In [57]:
SE9 = Cubic(eqn_SE9, line, sing_cubics_SE9.factor())

In [58]:
def apply_proj_to_eck(proj, eck):
    new_indices = []
    for i in range(len(eck)):
        new_indices.append(eck.index(eck[i]*proj)+1)
    return new_indices

In [60]:
[pl.labels for pl in SE9.tritangent_planes if pl.conditions == 0]

[['E1', 'G4', 'F14'],
 ['E2', 'G3', 'F23'],
 ['E3', 'G5', 'F35'],
 ['E4', 'G6', 'F46'],
 ['E5', 'G2', 'F25'],
 ['E6', 'G1', 'F16'],
 ['F12', 'F36', 'F45'],
 ['F13', 'F24', 'F56'],
 ['F15', 'F26', 'F34']]

In [61]:
labels = [sorted(pl.labels) for pl in SE9.tritangent_planes if pl.conditions == 0]
keys = list(SE9.cl_lines.keys())
labels

[['E1', 'F14', 'G4'],
 ['E2', 'F23', 'G3'],
 ['E3', 'F35', 'G5'],
 ['E4', 'F46', 'G6'],
 ['E5', 'F25', 'G2'],
 ['E6', 'F16', 'G1'],
 ['F12', 'F36', 'F45'],
 ['F13', 'F24', 'F56'],
 ['F15', 'F26', 'F34']]

In [62]:
adm_perm = SE9.find_admissible_permutations()

In [85]:
%time adm_proj = SE9.find_admissible_projectivities()

CPU times: user 5.6 s, sys: 216 ms, total: 5.81 s
Wall time: 49.5 s


In [86]:
simm_SE9 = SE9.find_simmetries(adm_proj)

In [87]:
len(simm_SE9)

54

In [89]:
perms = []
for simm in simm_SE9:
    perm = adm_perm[adm_proj.index(simm)]
    perms.append(Permutation([perm.index(label)+1 for label in keys]).to_cycles())

In [103]:
Gr = PermutationGroup(perms)

In [138]:
Gr.structure_description(latex=True)

'((C_{3} \\times C_{3}) \\rtimes C_{3}) \\rtimes C_{2}'

In [136]:
adm_perm.index(['E1', 'E2', 'E3', 'E4', 'E6', 'E5', 'G1', 'G2', 'G3', 'G4', 'G6', 'G5', 'F12', 'F13', 'F14', 'F16', 'F15', 'F23', 'F24', 'F26', 'F25', 'F34', 'F36', 'F35', 'F46', 'F45', 'F56'])

ValueError: ['E1', 'E2', 'E3', 'E4', 'E6', 'E5', 'G1', 'G2', 'G3', 'G4', 'G6', 'G5', 'F12', 'F13', 'F14', 'F16', 'F15', 'F23', 'F24', 'F26', 'F25', 'F34', 'F36', 'F35', 'F46', 'F45', 'F56'] is not in list

In [64]:
for triple in labels:
    print([adm_perm[200][keys.index(label)] for label in triple])

['E5', 'G2', 'F25']
['F14', 'E1', 'G4']
['F16', 'E6', 'G1']
['E3', 'G5', 'F35']
['F46', 'E4', 'G6']
['E2', 'G3', 'F23']
['F56', 'F24', 'F13']
['F45', 'F36', 'F12']
['F15', 'F26', 'F34']


In [65]:
for perm in adm_perm:
    for triple in labels:
        new_triple = []
        for label in triple:
            new_triple.append(perm[keys.index(label)])
        if sorted(new_triple) not in labels:
            print(False)

In [81]:
resulting_L_sets = []
for perm in adm_perm:
    perm_L_set = [perm[list(SE9.cl_lines.keys()).index(label)] for label in ['E1', 'G4', 'E2', 'G3', 'E3']]
    if perm_L_set not in resulting_L_sets:
        resulting_L_sets.append(perm_L_set)
    else:
        print(adm_perm.index(perm))

In [67]:
len(resulting_L_sets)

1296

In [69]:
SE9.cl_lines['E5'].plucker

(0, -2*c + 2*f, (-alpha + 1)*c + (alpha - 1)*f, (alpha + 1)*c + (alpha + 1)*f, -2*c - 2*f, -4*f)